In [ ]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [ ]:
rho = 11e3
Omega = 2*np.pi/86400
U0 = 8.7e-5
R = 3486e3

In [ ]:
def f(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2) + 4*np.cos(lat)**2*np.sin(alpha)**2
    
    if expr < 0:
        return 0.0

    sqrt_term = np.sqrt(expr)
    return sqrt_term * (np.sin(lat)**2 * np.cos(alpha)**2 + np.sin(alpha)**2)

In [ ]:
def Pflux(NoverOmega, lat):
 return quad(f, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
def f_trad(alpha, lat, NoverOmega):
    expr = (NoverOmega**2 - 1)*(1 - 4*np.sin(lat)**2)
    
    if expr < 0:
        return 0.0

    sqrt_term = np.sqrt(expr)
    return sqrt_term * (np.sin(lat)**2 * np.cos(alpha)**2 + np.sin(alpha)**2)

In [ ]:
def Pflux_trad(NoverOmega, lat):
 return quad(f_trad, 0, 2*np.pi, args=(lat, NoverOmega), limit=1000, epsabs=1e-10, epsrel=1e-10)[0]

In [ ]:
# Vectorize over latitude
latitudes = np.linspace(0, np.pi/2, 200) 
Pflux_vec = np.vectorize(lambda lat: Pflux(0.5, lat))
power_flux = Pflux_vec(latitudes)

latitudes = np.linspace(0, np.pi/2, 200) 
Pflux_trad_vec = np.vectorize(lambda lat: Pflux_trad(0.5, lat))
power_flux_trad = Pflux_trad_vec(latitudes)

In [ ]:
k0 = 2*np.pi/1e6
H = 0.7
hrms = 5e3

In [ ]:
# Plotting
plt.rc('text', usetex=True)
plt.plot(np.degrees(latitudes), (rho/8/np.pi**3)*(hrms**2*k0/(2*H-1))*(U0**2*Omega)*power_flux, label="exact")
plt.plot(np.degrees(latitudes), (rho/8/np.pi**3)*(hrms**2*k0/(2*H-1))*(U0**2*Omega)*power_flux_trad, label="traditional approximation")
plt.xlabel("Latitude (degrees)")
plt.ylabel(r'Power flux ($\mathrm{W/m^2}$)')
plt.title("Power flux vs Latitude")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Wrapper for latitude integration, including Jacobian (R^2 cos(lat)) and the 2*pi factor from integration over longitude
def integrand(lat, NoverOmega):
    return Pflux(NoverOmega, lat) * R**2 * 2*np.pi * np.cos(lat)

In [ ]:
N_values = np.linspace(0., 5.0, 41) 
results = []

for NoverOmega in N_values:
    total_flux, _ = quad(
        integrand,
        -np.pi/2,
        np.pi/2,
        args=(NoverOmega),
        limit=1000,
        epsabs=1e-10,
        epsrel=1e-10
    )
    results.append(total_flux)

In [ ]:
# Wrapper for latitude integration, including Jacobian (R^2 cos(lat)) and the 2*pi factor from integration over longitude
def integrand_trad(lat, NoverOmega):
    return Pflux_trad(NoverOmega, lat) * R**2 * 2*np.pi * np.cos(lat)

In [ ]:
N_values = np.linspace(0., 5.0, 41) 
results_trad = []

for NoverOmega in N_values:
    total_flux_trad, _ = quad(
        integrand_trad,
        -np.pi/2,
        np.pi/2,
        args=(NoverOmega),
        limit=1000,
        epsabs=1e-10,
        epsrel=1e-10
    )
    results_trad.append(total_flux_trad)

In [ ]:
ImKcmbtarget = -1.85e-5

In [ ]:
As = 5.8531e34
Af = 9.0583e36
A  = 8.0115e37
Am = A - (As+Af)
wf = 2.4e-11

In [ ]:
k0 = 2*np.pi/1e6
H = 0.7
hrms = 5.67e3

In [ ]:
plt.rc('text', usetex=True)

# Secondary y-axis
 
ax = plt.gca()
ax.secondary_yaxis('right', functions=(lambda y: y / sf, lambda y: y * sf))\
         .set_ylabel(r'$\mathrm{Im}(K_\mathrm{cmb})$', fontsize=14)
ax.axhline(y=sf*ImKcmbtarget, color='k', linestyle='--')

ax.set_xlim(N_values[0] - 0.25, N_values[-1] + 0.25)
x_fill = np.linspace(N_values[0] - 0.25, N_values[-1] + 0.25, 100)
ax.fill_between(x_fill, sf*(-1.71e-5), sf*(-1.99e-5), color='gray', alpha=0.2)

plt.plot(N_values,(rho/8/np.pi**3)*(hrms**2*k0*H/(2*H-1))*(U0**2*Omega)*np.array(results), marker='o', linestyle='-', label="exact")
plt.plot(N_values,(rho/8/np.pi**3)*(hrms**2*k0*H/(2*H-1))*(U0**2*Omega)*np.array(results_trad), marker='o', linestyle='-', label="traditional approximation")
plt.xlabel(r'$N/\Omega$', fontsize=14)
plt.ylabel(r'Total power (W)', fontsize=14)
plt.legend()
plt.grid(True)


plt.savefig("Total_power.pdf")
plt.show()

In [ ]:
plt.plot(np.array(results)/(R**2 * 2*np.pi), marker='o', linestyle='-', label="exact")